# OpenCV Face Detection HDMI

In this notebook, opencv face detection will be applied to HDMI input images.

To run all cells in this notebook a HDMI input source and HDMI output monitor are required.  

References:

https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml

### Step 1: Load the overlay

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.ov5640 import OV5640
from pynq.lib.iic import *
from pynq.lib.video import *

base = BaseOverlay("base.bit")
camera_in = base.video.camera_in
hdmi_out  = base.video.hdmi_out

## Instantiate I2C and configure OV5640 Camera

In [ ]:
iic = AxiIIC(base.ip_dict['cam_iic'])
# OV5640器件地址
address = 0x3c  
ov5640= OV5640(address, iic)

### 初始化OV5640

In [ ]:
ov5640.init()

### Step 2: Initialize HDMI I/O

In [ ]:
camera_in.configure(PIXEL_RGB)
hdmi_out.configure(camera_in.mode, PIXEL_RGB)

camera_in.start()
hdmi_out.start()

In [ ]:
camera_in.tie(hdmi_out)

### Step 3: Show input frame using IPython Image
Source: http://gardnerproductions.ca/wp-content/uploads/2015/11/bigstock-People-with-different-emotions-930x701-thegem-blog-default.jpg

In [ ]:
import PIL.Image
frame = camera_in.readframe()
img = PIL.Image.fromarray(frame)
img.save("/home/xilinx/jupyter_notebooks/base/video/data/face_detect.jpg")

img

### Step 4: Apply the face detection to the input

In [ ]:
import cv2
import numpy as np

frame = camera_in.readframe()

face_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_eye.xml')

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

for (x,y,w,h) in faces:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = frame[y:y+h, x:x+w]

    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

### Step 5: Show results on HDMI output
Output OpenCV results via HDMI.

In [ ]:
hdmi_out.writeframe(frame)

### Step 6: Show results within notebook
Output OpenCV results as JPEG.

In [ ]:
img = PIL.Image.fromarray(frame)
img.save("/home/xilinx/jupyter_notebooks/base/video/data/face_detect.jpg")

img

### Step 7: Release HDMI

In [ ]:
hdmi_out.stop()
camera_in.stop()
del camera_in, hdmi_out